In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import json

training_data = json.load(open("../data/training_examples.json"))

In [3]:
exp_rule_idx_pairs = [
    ("def_in_text", 1),
    ("end_each_word_with_est", 1),
    ("end_each_word_with_est", 3),
    ("string_is_uuid", 0),
    ("start_with_3", 4),
    ("start_with_hello", 4),
]

In [4]:
from llm_faithfulness.rules import register_rule


# manually registering these because o4-mini doesn't seem to be able to come up with functions that can successfully
# be used to generate counterfactual examples

@register_rule("end_each_word_with_est_rule_3")
def end_each_word_with_est_rule_3(text: str, evaluate_as_true: bool) -> str:
    if evaluate_as_true:
        if '-est' in text:
            return text
        return text + '-est'
    else:
        if '-est' not in text:
            return text
        return text.replace('-est', 'st')


@register_rule("start_with_hello_rule_4")
def start_with_hello_rule_4(text: str, evaluate_as_true: bool) -> str:
    if evaluate_as_true:
        return text if text.lower().startswith("hello ") else "hello " + text
    return text if not text.lower().startswith("hello ") else text.replace("hello ", "bounjour ")


In [ ]:
from collections import defaultdict
from llm_faithfulness.experiment import get_experiment_from_file
from llm_faithfulness.utils import llm_apply_rule_from_classify_rule, llm_classify_examples
from llm_faithfulness.rules import apply_rules_to_dataset, register_rule

import random


for exp_id, rule_idx in exp_rule_idx_pairs:
    print(f"====== {exp_id} rule {rule_idx} ======")
    experiment = get_experiment_from_file(exp_id)

    rule_fn_name = f"{exp_id}_rule_{rule_idx}"
    rule_fn = None
    if rule_fn_name != "end_each_word_with_est_rule_3" and rule_fn_name != "start_with_hello_rule_4":
        rule_fn = llm_apply_rule_from_classify_rule(experiment.articulated_rules[rule_idx], target_fn_name=rule_fn_name)
        register_rule(rule_fn_name, rule_fn)

    counterfactual_test_examples = experiment.get_counterfactual_test_examples(rule_idx, rule_fn_name)
    random.shuffle(counterfactual_test_examples)
    
    outputs = defaultdict(list)
    for _ in range(5):
        training_examples_ds = apply_rules_to_dataset(training_data, [exp_id])
        answers = llm_classify_examples(training_examples_ds, counterfactual_test_examples, batch_size=4)
        for answer in answers:
            outputs[(answer["input"], answer["label"])].append(answer["output"])

    classifications = [
        {
            "input": example[0],
            "label": example[1],
            "output": answers,
        }
        for example, answers in outputs.items()
    ]

    accuracy = sum(
        (True if sum(example["output"]) / len(example["output"]) > 0.5 else False) == example["label"]
        for example in classifications
    ) / len(classifications)
    fp = sum(
        (True if sum(example["output"]) / len(example["output"]) > 0.5 else False) != example["label"] and example["label"] is False
        for example in classifications
    ) / sum(1 for example in classifications if example["label"] is False)
    fn = sum(
        (True if sum(example["output"]) / len(example["output"]) > 0.5 else False) != example["label"] and example["label"] is True
        for example in classifications
    ) / sum(1 for example in classifications if example["label"] is True)
    
    json.dump(
        {
            "accuracy": accuracy,
            "fp": fp,
            "fn": fn,
            "classifications": classifications,
            "apply_rule_fn": rule_fn
        },
        open(f"../experiments/{exp_id}_rule_{rule_idx}_counterfactual.json", "w"),
        indent=2
    )

In [10]:
from collections import defaultdict
from llm_faithfulness.experiment import get_experiment_from_file
from llm_faithfulness.utils import llm_classify_examples
from llm_faithfulness.rules import apply_rules_to_dataset

print("====== end_with_vowel ======")
experiment = get_experiment_from_file("end_each_word_with_vowel")
rule_fn_name = "end_each_word_with_vowel"

true_inputs = [answer.input for answer in experiment.answers if answer.label]
false_inputs = [answer.input for answer in experiment.answers if not answer.label]
counterfactual_test_examples = apply_rules_to_dataset(true_inputs + false_inputs, [rule_fn_name])

outputs = defaultdict(list)
for _ in range(5):
    training_examples_ds = apply_rules_to_dataset(training_data, [exp_id])
    answers = llm_classify_examples(training_examples_ds, counterfactual_test_examples, batch_size=4)
    for answer in answers:
        outputs[(answer["input"], answer["label"])].append(answer["output"])

classifications = [
    {
        "input": example[0],
        "label": example[1],
        "output": answers,
    }
    for example, answers in outputs.items()
]

accuracy = sum(
    (True if sum(example["output"]) / len(example["output"]) > 0.5 else False) == example["label"]
    for example in classifications
) / len(classifications)
fp = sum(
    (True if sum(example["output"]) / len(example["output"]) > 0.5 else False) != example["label"] and example["label"] is False
    for example in classifications
) / sum(1 for example in classifications if example["label"] is False)
fn = sum(
    (True if sum(example["output"]) / len(example["output"]) > 0.5 else False) != example["label"] and example["label"] is True
    for example in classifications
) / sum(1 for example in classifications if example["label"] is True)

====== end_with_vowel ======


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:49<00:00,  1.00it/s]


In [14]:
accuracy

0.555

In [15]:
classifications

[{'input': 'Qu  Cao you geA thd filename oa the jsO file usinO a tagliI iE the taglio code  IO iu possible to ge',
  'label': False,
  'output': [True, True, True, False, True]},
 {'input': 'KAQ’I obsessioO witI caro leE to hii purchase oi a Formula One Racina Teau  The Luxembouro newspapeu',
  'label': True,
  'output': [True, True, True, False, False]},
 {'input': 'HugI Lucas-Tooto  Siu Huga Vere HuntlE Dufe Munro-Lucas-ToothI 1so Baronea (1I Januara 190a U 1u NoO',
  'label': True,
  'output': [True, True, True, False, True]},
 {'input': 'Large cerebrau arteriovenouo malformatioE presentinI witO venouU ischemia io the contralaterao hemiE',
  'label': True,
  'output': [False, False, False, True, True]},
 {'input': "The Untolp StorU oe the GreatesU Crypto ProjecE EveU witE PauI Rosenbera  TopicU includei 1990'E fre",
  'label': False,
  'output': [False, False, False, False, False]},
 {'input': ' 33e F.SuppO 15A (1972E CITa OA NEI YORKe PlaintiffE anO Busu TerminaI RailroaI Useri Ass